<a href="https://colab.research.google.com/github/Serena-Anthony/vat-score-calculation/blob/main/vat_score.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install Python libraries
!pip install --quiet \
    opencv-python-headless \
    moviepy \
    pytesseract \
    textstat \
    nltk \
    pydub \
    SpeechRecognition \
    ffmpeg-python \
    numpy \
    librosa \
    torch \
    torchvision \
    "httpx>=0.23.0,<1.0.0"

# Install system dependencies
!apt-get update -y
!apt-get install -y tesseract-ocr ffmpeg


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.3/105.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [5]:

# ------------------------- 📁 CHOOSE INPUT MODE -------------------------
import os
from google.colab import drive
from google.colab import files

use_drive = input("Use Google Drive? (yes/no): ").strip().lower()

if use_drive == 'yes':
    drive.mount('/content/drive')
    base_path = input("Enter full path to your video folder in Drive (e.g., /content/drive/MyDrive/material): ").strip()
else:
    upload_path = "/content/uploaded_videos"
    os.makedirs(upload_path, exist_ok=True)
    uploaded = files.upload()
    for fname in uploaded.keys():
        os.rename(fname, os.path.join(upload_path, fname))
    base_path = upload_path

print(f"✅ Scanning folder: {base_path}")


Use Google Drive? (yes/no): no


Saving food_intro_1.mp4 to food_intro_1.mp4
✅ Scanning folder: /content/uploaded_videos


In [6]:
# ------------------------- 🧠 IMPORTS -------------------------
import librosa
import cv2
import pandas as pd
import numpy as np
import torch
from moviepy.editor import VideoFileClip
import torchvision.transforms as transforms
import torchvision.models as models
import tempfile

import pandas as pd
import numpy as np
import librosa
from moviepy.editor import VideoFileClip
import tempfile
import textstat

  if event.key is 'enter':



EDITSSSS


In [ ]:

# Load pretrained ResNet-18
resnet_model = resnet18(pretrained=True)
resnet_model.eval()

# Image transform for ResNet
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],  # ImageNet mean
        std=[0.229, 0.224, 0.225]    # ImageNet std
    )
])

In [ ]:
# ------------------------- 📊 SCORING FUNCTIONS -------------------------

def get_audio_score(video_path):
    """Auditory score based on RMS energy of audio."""
    try:
        with tempfile.NamedTemporaryFile(suffix=".wav") as tmp_audio:
            videoclip = VideoFileClip(video_path)
            audioclip = videoclip.audio
            audioclip.write_audiofile(tmp_audio.name, verbose=False, logger=None)
            y, sr = librosa.load(tmp_audio.name, sr=None)
            rms = librosa.feature.rms(y=y)
            score = np.mean(rms) * 1000  # Scale to 0-100 range
            return round(min(score, 100.0), 2)
    except Exception as e:
        print(f"❌ Audio score failed for {video_path}: {e}")
        return 0.0

def get_visual_score(video_path):
    """Visual score: duration in seconds normalized to 0–100."""
    try:
        clip = VideoFileClip(video_path)
        duration = clip.duration
        score = min(duration / 10.0, 100.0)  # e.g., 100s duration = score 100
        return round(score, 2)
    except Exception as e:
        print(f"❌ Visual score failed for {video_path}: {e}")
        return 0.0

def get_textual_score(video_path):
    """Textual score using FPS as proxy for slide/text density."""
    try:
        clip = VideoFileClip(video_path)
        fps = clip.fps
        return round(min(fps, 100.0), 2)
    except Exception as e:
        print(f"❌ Textual score failed for {video_path}: {e}")
        return 0.0


In [ ]:
# ------------------------- 📁 RECURSIVELY SCAN FOLDER -------------------------

results = []

for root, dirs, files in os.walk(base_path):
    for file in files:
        file_path = os.path.join(root, file)
        rel_path = os.path.relpath(file_path, base_path)

        print(f"🔍 Processing: {rel_path}")

        if file.endswith(".mp4"):
            visual = get_visual_score(file_path)
            auditory = get_audio_score(file_path)
            textual = get_textual_score(file_path)
            flesch = 0.0
        elif file.endswith(".txt"):
            visual = 0.0
            auditory = 0.0
            textual = 1.0
            with open(file_path, "r", encoding="utf-8") as f:
                content = f.read()
            flesch = calculate_readability_metrics(content)
        else:
            continue

        results.append({
            "File": rel_path,
            "Visual": visual,
            "Auditory": auditory,
            "Textual": textual,
            "FleschScore": flesch
        })


In [ ]:
# ------------------------- 📊 DISPLAY DATA -------------------------
df_scores = pd.DataFrame(results)
df_scores = df_scores[["File", "Visual", "Auditory", "Textual", "FleschScore"]]
print("\n📊 Final Scores DataFrame:")
print(df_scores)


old

In [ ]:
# ✅ Step 2: Imports
import os
import librosa
import cv2
import pandas as pd
import numpy as np
import torch
from moviepy.editor import VideoFileClip
import torchvision.transforms as transforms
import torchvision.models as models
import tempfile

# ✅ Step 3: Set path to your Drive folder
materials_path = "/content/drive/MyDrive/material"

# ✅ Step 4: Scoring functions

def get_visual_score(video_path):
    try:
        model = models.resnet18(pretrained=True)
        model.eval()

        transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
        ])

        cap = cv2.VideoCapture(video_path)
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        scores = []
        frame_interval = max(1, frame_count // 10)

        for i in range(0, frame_count, frame_interval):
            cap.set(cv2.CAP_PROP_POS_FRAMES, i)
            ret, frame = cap.read()
            if ret:
                img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                input_tensor = transform(img).unsqueeze(0)
                with torch.no_grad():
                    output = model(input_tensor)
                    score = torch.nn.functional.softmax(output[0], dim=0).max().item()
                    scores.append(score)
        cap.release()
        return round(np.mean(scores) * 100, 2)
    except Exception as e:
        print(f"❌ Visual processing failed for {video_path}: {e}")
        return 0.0

def get_audio_score(video_path):
    try:
        with tempfile.NamedTemporaryFile(suffix=".wav") as tmp_audio:
            videoclip = VideoFileClip(video_path)
            audioclip = videoclip.audio
            audioclip.write_audiofile(tmp_audio.name, verbose=False, logger=None)
            y, sr = librosa.load(tmp_audio.name, sr=None)
            rms = librosa.feature.rms(y=y)
            score = np.mean(rms) * 1000  # Scaled RMS value
            return round(min(score, 100.0), 2)
    except Exception as e:
        print(f"❌ Audio processing failed for {video_path}: {e}")
        return 0.0

def get_textual_score(video_path):
    try:
        clip = VideoFileClip(video_path)
        duration = clip.duration
        fps = clip.fps
        frame_count = int(duration * fps)
        return round(min(frame_count / duration, 100.0), 2)  # proxy for text/slide density
    except Exception as e:
        print(f"❌ Textual processing failed for {video_path}: {e}")
        return 0.0

# ✅ Step 5: Recursively scan videos and score
results = []

for root, dirs, files in os.walk(materials_path):
    for file in files:
        if file.endswith(".mp4"):
            file_path = os.path.join(root, file)
            rel_path = os.path.relpath(file_path, materials_path)
            print(f"🎥 Processing video: {rel_path}")
            visual = get_visual_score(file_path)
            auditory = get_audio_score(file_path)
            textual = get_textual_score(file_path)
            results.append({
                "File": rel_path,
                "Visual": visual,
                "Auditory": auditory,
                "Textual": textual,
            })

# ✅ Step 6: Create DataFrame and display
df_scores = pd.DataFrame(results)
print(df_scores)

# Optionally save
# df_scores.to_csv("video_scores.csv", index=False)


  if event.key is 'enter':

  warnings.warn(

  warnings.warn(msg)



🎥 Processing video: nutrients2.mp4


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 135MB/s]
  warnings.warn(

  warnings.warn(msg)



🎥 Processing video: nutrients3.mp4
🎥 Processing video: food_intro_3.mp4
               File  Visual    Auditory  Textual
0    nutrients2.mp4   40.46  100.000000     30.0
1    nutrients3.mp4   27.40   57.119999     30.0
2  food_intro_3.mp4   28.70   45.450001     30.0


*code for checking if cn calculate content from video*

checking including ip and readability

In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [ ]:
!pip install textstat

In [ ]:
import textstat

def calculate_readability_metrics(text):
    try:
        flesch = textstat.flesch_reading_ease(text)
        return round(flesch, 2)
    except Exception as e:
        print(f"❌ Readability calc failed: {e}")
        return 0.0, 0.0


In [ ]:
import pytesseract

def extract_text_from_frames(video_path, frame_interval=30):
    try:
        cap = cv2.VideoCapture(video_path)
        text_data = ""
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        for i in range(0, frame_count, frame_interval):
            cap.set(cv2.CAP_PROP_POS_FRAMES, i)
            ret, frame = cap.read()
            if ret:
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
                text = pytesseract.image_to_string(thresh)
                text_data += text + " "

            # print(text_data)
        cap.release()
        return text_data.strip()
    except Exception as e:
        print(f"❌ OCR failed: {e}")
        return ""


In [ ]:
# ✅ Step 5: Recursively scan videos and score (including Readability and IP)
results = []

for root, dirs, files in os.walk(materials_path):
    for file in files:
        if file.endswith(".mp4"):
            file_path = os.path.join(root, file)
            rel_path = os.path.relpath(file_path, materials_path)
            print(f"🎥 Processing video: {rel_path}")

            # Compute Visual, Audio, and Textual Scores
            visual = get_visual_score(file_path)
            auditory = get_audio_score(file_path)
            textual = get_textual_score(file_path)

            # OCR Text Extraction and Readability/IP Scores
            ocr_text = extract_text_from_frames(file_path)
            flesch, ip = calculate_readability_metrics(ocr_text)

            # Store result
            results.append({
                "File": rel_path,
                "Visual": visual,
                "Auditory": auditory,
                "Textual": ocr_text,
                "FleschScore": flesch,
                "InfoProcessing": ip
            })

# ✅ Step 6: Create DataFrame and display
df_scores = pd.DataFrame(results)
print(df_scores)

  warnings.warn(

  warnings.warn(msg)



🎥 Processing video: nutrients2.mp4


  warnings.warn(

  warnings.warn(msg)



🎥 Processing video: nutrients3.mp4


  warnings.warn(

  warnings.warn(msg)



🎥 Processing video: food_intro_3.mp4
               File  Visual    Auditory  \
0    nutrients2.mp4   40.46  100.000000   
1    nutrients3.mp4   27.40   57.119999   
2  food_intro_3.mp4   28.70   45.450001   

                                             Textual  FleschScore  \
0  learnFatafat:\n\n> allagy\n\n \n learnFatafat...        53.89   
1  Pe bb Ie)\n\n \n     \n\nThe three key food gr...        61.29   
2  =p\n\nae ie pebbles.in\n\n6". CBSE - Food: Whe...        91.61   

   InfoProcessing  
0            8.57  
1            1.85  
2            5.56  


In [ ]:
results = []

for root, dirs, files in os.walk(materials_path):
    for file in files:
        if file.endswith((".mp4", ".txt")):
            file_path = os.path.join(root, file)
            rel_path = os.path.relpath(file_path, materials_path)

            print(f"📁 Processing: {rel_path}")

            if file.endswith(".mp4"):
                # 🎥 For video files
                visual = get_visual_score(file_path)
                auditory = get_audio_score(file_path)
                textual = get_textual_score(file_path)

                # Extract text from video frames and calculate metrics
                ocr_text = extract_text_from_frames(file_path)
                flesch, ip = calculate_readability_metrics(ocr_text)

            elif file.endswith(".txt"):
                # 📄 For text files
                visual = 0.0
                auditory = 0.0
                textual = 1.0

                # Read file content and calculate metrics
                with open(file_path, "r", encoding="utf-8") as f:
                    text_content = f.read()
                flesch, ip = calculate_readability_metrics(text_content)

            # Store results
            results.append({
                "File": rel_path,
                "Visual": round(visual, 2),
                "Auditory": round(auditory, 2),
                "Textual": round(textual, 2),
                "FleschScore": round(flesch, 2),
                "InfoProcessing": round(ip, 2)
            })

# ✅ Step 6: Create DataFrame and display
df_scores = pd.DataFrame(results)
df_scores = df_scores[["File", "Visual", "Auditory", "Textual", "FleschScore", "InfoProcessing"]]
print(df_scores)

# Optional: Save to CSV
# df_scores.to_csv("/content/drive/MyDrive/video_scores.csv", index=False)


  warnings.warn(

  warnings.warn(msg)



📁 Processing: nutrients3.mp4


  warnings.warn(

  warnings.warn(msg)



📁 Processing: nutrients2.mp4


  warnings.warn(

  warnings.warn(msg)



📁 Processing: food_intro_3.mp4
               File  Visual    Auditory  Textual  FleschScore  InfoProcessing
0    nutrients3.mp4   27.40   57.119999     30.0        61.29            1.85
1    nutrients2.mp4   40.46  100.000000     30.0        53.89            8.57
2  food_intro_3.mp4   28.70   45.450001     30.0        91.61            5.56
